In [1]:
import requests
from datetime import datetime, timedelta
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
ACCESS_TOKEN = '5816254b5816254b5816254b7558614893558165816254b386fa62025fb312bc944f6cc'
APP_ID = 7826904
API_URL = 'https://api.vk.com/method'
V = '5.91'

In [3]:
def get_post(wall_id, post_id):
    api_method = f'{API_URL}/wall.getById'
    resp = requests.get(api_method, params={
        'access_token': ACCESS_TOKEN,
        'posts': f'{wall_id}_{post_id}',
        'v': V
    })
    data = resp.json()['response']
    return data
get_post(-178613088, 34223)

[{'id': 34223,
  'from_id': -178613088,
  'owner_id': -178613088,
  'date': 1641736680,
  'marked_as_ads': 0,
  'post_type': 'post',
  'text': 'Васильевский остров',
  'attachments': [{'type': 'photo',
    'photo': {'album_id': -7,
     'date': 1641678892,
     'id': 457267362,
     'owner_id': -178613088,
     'access_key': 'da70e5b5743405d72a',
     'sizes': [{'height': 130,
       'url': 'https://sun9-35.userapi.com/impg/AAlgrZ82RNNXnGqgAode6A6NVIiXI72P4ti_kw/OkuL4HadNNA.jpg?size=105x130&quality=96&sign=7bae08ab6d1908e60232d6adc314b05b&c_uniq_tag=SIni51TTHCBtbio6pH_ZOkHjU0P1mCFuZ4EXXlTIRKc&type=album',
       'type': 'm',
       'width': 105},
      {'height': 161,
       'url': 'https://sun9-35.userapi.com/impg/AAlgrZ82RNNXnGqgAode6A6NVIiXI72P4ti_kw/OkuL4HadNNA.jpg?size=130x161&quality=96&sign=862fed6c6ba9e088c7cd91a5f44a1f8f&c_uniq_tag=5OxD-FCGBq_uxoMx_RYvgJJ5yqvtIGJ3zAH4H3gJbB0&type=album',
       'type': 'o',
       'width': 130},
      {'height': 247,
       'url': 'https://sun

In [4]:
def search_posts(theme, start, final):
    delta = timedelta(hours=1)
    
    api_method = f'{API_URL}/newsfeed.search'
    data = []
    data_temp = []
    
    while start < final:
        end = start + delta
        start_from = ''
        f = True
        while f:
            params={
                'access_token': ACCESS_TOKEN,
                'count': 200,
                'start_time': time.mktime(start.timetuple()),
                'end_time': time.mktime(end.timetuple()),
                'q': theme,
                'v': V
            }
            if start_from!='':
                params['start_from'] = start_from
            resp = requests.get(api_method, params)
            data += resp.json()['response']['items']
            f = 'next_from' in resp.json()['response']
            if f:
                start_from = resp.json()['response']['next_from']
        start = end
        data_temp.append(len(data))
    return data

In [6]:
data = []
start = datetime(2022,8,11)
end = datetime(2022,8,12)
delta = timedelta(hours=1)


res = search_posts('COVID-19', start, end)
    
print(len(res))
res = list(filter(lambda x: x['post_type'] == 'post' and x['comments'] != '0', res))
for d in res:
    if 'attachments' in d: del d['attachments']
    del d['post_type']
    d['date'] = datetime.fromtimestamp(d['date'])
    if 'marked_as_ads' in d: del d['marked_as_ads']
    if 'views' in d: d['views'] =  d['views']['count']
    if 'likes' in d: d['likes'] =  d['likes']['count']
    if 'reposts' in d: d['reposts'] =  d['reposts']['count']
    if 'comments' in d: d['comments'] =  d['comments']['count']
print(len(res))
res

4581
4581


[{'id': 99387,
  'date': datetime.datetime(2022, 8, 11, 1, 0),
  'owner_id': -102226711,
  'from_id': -102226711,
  'text': 'Премьер-министр Японии Фумио Кисида утвердил новый состав правительства после отставки предыдущего кабинета министров. Как сообщает Kyodo News, ожидаемо свои посты сохранили главы МИД и Минфина Ёсимаса Хаяси и Сюнъити Судзуки.\n \nГенеральным секретарем остался Хирокадзу Мацуно. \n \nТэцуо Сайто продолжит занимать пост министра государственных земель, транспорта, инфраструктуры и туризма. \n \nЯсукадзу Хамада, занимавший пост министра обороны с 2008 по 2009 год, вернулся на этот пост, заменив Нобуо Киси. Киси, младший брат убитого экс-премьера Синдзо Абэ станет советником премьер-министра по вопросам национальной безопасности. Задержанный за убийство Абэ обвинял бывшего премьера в связях с организацией "Церковь объединения", к которой у стрелявшего были личные претензии. Kyodo напоминает, что формально Киси был смещен с поста министра по состоянию здоровья, однак

In [7]:
df = pd.DataFrame(res, columns=['date', 'id', 'owner_id', 'from_id', 'text', 'comments', 'likes', 'reposts', 'views'])
df = df[df['comments'] > 0]
# df = df.append(res, ignore_index=True)
# df.text.to_csv('all.csv')

# df.text.str.len().sort_values(ascending=False)
# s = df.text.str.len().sort_values(ascending=False).index
# df = df.reindex(s)
# # df.text.to_csv('long_texts.csv')
# df.text.str.len().sort_values(ascending=False)
# # print(df.reindex(s))
# x = df.text.str.len()
# df.sort_values(by=['comments']).tail(50)
df.to_csv('2021-06-11-12.csv')
df

,date,id,owner_id,from_id,text,comments,likes,reposts,views
3,2022-08-11 00:57:18,193753,633372150,633372150,>HOLOCAUSTE DES VACCINS CONTRE LE COVID-19< \n...,1,53,15,817.0
7,2022-08-11 00:52:58,324053,-66772954,-66772954,Российские военные выдают все новые порции раз...,2,25,10,542.0
13,2022-08-11 00:49:14,13040,623459561,623459561,🇨🇭 👍🏼👍🏼👍🏼😁Les plus de 80 ans boudent le deuxiè...,9,27,5,247.0
22,2022-08-11 00:39:23,1446,331745341,331745341,,2,9,0,4045.0
25,2022-08-11 00:37:19,383053,-27075486,-27075486,💉📈 В Петербурге продолжает расти спрос на прох...,11,10,3,4663.0
...,...,...,...,...,...,...,...,...,...
4531,2022-08-11 23:28:18,6993,3450829,3450829,В Подмосковье начала расти статистика вызовов ...,6,18,0,250.0
4553,2022-08-11 23:15:00,2834861,-166856301,-166856301,,35,35,16,15763.0
4557,2022-08-11 23:13:29,7349,35481218,35481218,❌ОТ ЭТОГО ВЫПАДАЮТ ВОЛОСЫ ❌\nКрайне наболевшая...,8,48,5,2273.0
4571,2022-08-11 23:06:36,6205,-80075684,-80075684,"У 3 граждан КНР, прилетевших из РФ в Пекин, вы...",2,11,0,1481.0


# Комментарии

In [109]:
def get_all_comments_from_post(wall_id, post_id):
    # Cобирает 100 комментарев с поста
    # Проверяет нужно ли больше
    # Проверяет нужно ли дополнительные ответные комментарии для комментариев
    # Обходит ограничение в 5 запросов в сек.
    
    res = []
    
    last_time = datetime.now()
    count_thread_call = 1
    
    data = get_portition_of_comments(wall_id, post_id)
    text = [i['text'] for i in data['items']]
    print(text)
    
#     if data['current_level_count'] > 2:
# #         while (data['real_offset']):
            
#         add = get_portition_of_comments(wall_id, post_id, data['items'][-1]['id'])
#         text = [i['text'] for i in add['items']]
#         res.append(text)
#     else:
#         text = [i['text'] for i in data['items']]
#         res.append(text)
#     print(res, len(add['items']))
    
#     res = []
    
#     for comment in data['items']:
#         thread = [sub_comment['text'] for sub_comment in comment['thread']['items']]
#         if comment['thread']['count']>10:
#             if count_thread_call == 5:
#                 while (datetime.now() - last_time <= timedelta(seconds=1)):
#                     pass
#                 last_time = datetime.now()
#                 count_thread_call = 0
            
#             resp = requests.get(api_method, params={
#                 'access_token': ACCESS_TOKEN,
#                 'owner_id': f'{wall_id}',
#                 'post_id': f'{post_id}',
#                 'preview_length': 0,
#                 'comment_id': comment['id'],
#                 'v': V
#             })
#             count_thread_call += 1
#             print('?!!!!!!!!!!', count_thread_call, '!!!!!!!!!!!!!', )
#             thread = resp.json()['response']
#         res.append((comment['text'], thread))
    return data

def get_portition_of_comments(wall_id, post_id, start_comment_id='', thread_id=''):
    #Получает порцию комментариев с указаного треда
    api_method = f'{API_URL}/wall.getComments'
    resp = requests.get(api_method, params={
        'access_token': ACCESS_TOKEN,
        'count': 100,
        'owner_id': f'{wall_id}',
        'post_id': f'{post_id}',
        'preview_length': 0,
        'thread_items_count': 10,
        'comment_id': thread_id,
        'start_comment_id': start_comment_id,
        'need_likes': 1,
        'v': V
    })
    resp = resp.json()['response']        
    return resp

c = 1
# data = get_all_comments_from_post(-173277106, 3213574)
data = get_all_comments_from_post(-18895718, 1769277)
# data[0]
# print(data)
# for comments in data:
#     print(c, comments)
#     c+=1
#     for sub_comment in comments[1]['items']:
#         print('\t', sub_comment)
#         print(comments[1])        

['Было же 78%,если,не ошибаюсь) куда делись 25%???', 'счастье-то какое 😄😄😄😄😄😄', 'и чё?', 'а на НГ будет синий уровень', 'Очередное гонево.', 'Ага балаболы', 'Пизд.ж.']
